In [55]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import confusion_matrix,classification_report
from xgboost import XGBClassifier
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
import time

ALL = False
NO_SMOTE = True

# 判断 processed 文件夹是否存在
import os
if not os.path.exists('../data/processed'):
    print("Creating processed data folder...")
    # 读取CSV文件
    train_data = pd.read_csv('../data/raw/trainSet_res_with_distances.csv', dtype={'msisdn': 'str'})
    train_labels = pd.read_csv('../data/raw/trainSet_ans.csv', dtype={'msisdn': 'str'})

    validation_data = pd.read_csv('../data/raw/validationSet_res_with_distances.csv', dtype={'msisdn': 'str'})


    # 遍历 groupby('msisdn') 的结果，对每个 msisdn 进行数据增强
    # ------
    from tqdm import tqdm
    import os
    import sys
    sys.path.append(os.path.join(os.path.dirname('./'), '../'))
    from utils.augmentation import Augmentation

    addition_train_data = []
    addition_train_labels = []

    times = 4
    ratio_range = 0.1
    pbar = tqdm(train_data.groupby('msisdn'))
    for msisdn, group in pbar:
        if msisdn == 0:
            continue
        # print(f"Augmenting msisdn {msisdn}")
        pbar.set_description(f"Augmenting msisdn {msisdn}")
        label = train_labels[train_labels['msisdn'] == msisdn].iloc[0]['is_sa']
        aug = Augmentation(group, label, 'msisdn', 'is_sa')
        # 对正负样本进行平衡 样本比 1:4
        if label == 1:
            res_df, res_labels = aug.times(ratio=ratio_range, times=3 + times * 4, method='mask')
        else:
            res_df, res_labels = aug.times(ratio=ratio_range, times=times, method='mask')
        addition_train_data.append(res_df)
        addition_train_labels.append(res_labels)
    addition_train_data = pd.concat(addition_train_data)
    addition_train_labels = pd.concat(addition_train_labels)

    # 将新数据加入到train_data中
    train_data = pd.concat([train_data, addition_train_data], ignore_index=True).reset_index(drop=True)
    train_labels = pd.concat([train_labels, addition_train_labels], ignore_index=True).reset_index(drop=True)
    # 按照 msisdn, start_time 排序
    sort_start_time = time.time()
    train_data = train_data.sort_values(by=['msisdn', 'start_time']).reset_index(drop=True)
    train_labels = train_labels.sort_values(by=['msisdn']).reset_index(drop=True)
    print('sort time:', time.time() - sort_start_time)
    # ------------------

    # save
    print("Saving processed data...")
    os.makedirs('../data/processed', exist_ok=True)
    train_data.to_csv('../data/processed/train_data.csv', index=False)
    train_labels.to_csv('../data/processed/train_labels.csv', index=False)
    validation_data.to_csv('../data/processed/validation_data.csv', index=False)

else:
    print("Reading processed data...")
    train_data = pd.read_csv('../data/processed/train_data.csv', dtype={'msisdn': 'str'})
    train_labels = pd.read_csv('../data/processed/train_labels.csv', dtype={'msisdn': 'str'})

    # 读取验证集
    validation_data = pd.read_csv('../data/processed/validation_data.csv', dtype={'msisdn': 'str'})


# 转换时间格式
train_data['start_time'] = pd.to_datetime(train_data['start_time'], format='%Y%m%d%H%M%S')
train_data['end_time'] = pd.to_datetime(train_data['end_time'], format='%Y%m%d%H%M%S')
train_data['open_datetime'] = pd.to_datetime(train_data['open_datetime'], format='%Y%m%d%H%M%S')
train_data['update_time'] = pd.to_datetime(train_data['update_time'])
train_data['date'] = pd.to_datetime(train_data['date'])

validation_data['start_time'] = pd.to_datetime(validation_data['start_time'], format='%Y%m%d%H%M%S')
validation_data['end_time'] = pd.to_datetime(validation_data['end_time'], format='%Y%m%d%H%M%S')
validation_data['open_datetime'] = pd.to_datetime(validation_data['open_datetime'], format='%Y%m%d%H%M%S',errors='coerce')
validation_data['update_time'] = pd.to_datetime(validation_data['update_time'])
validation_data['date'] = pd.to_datetime(validation_data['date'])

Reading processed data...


In [56]:
len(train_data), len(validation_data)

(4132071, 232250)

In [57]:
# 为每条记录添加start_time_diff，记录 start_time 与上一条记录的 start_time 之差 (单位：秒)
start_time_diff = train_data.groupby('msisdn')['start_time'].diff().dt.total_seconds().fillna(0).reset_index(drop=True)
# 将该列加入到数据集中
train_data['start_time_diff'] = start_time_diff.copy()
# time_diff_start2end = train_data.groupby('msisdn')['end_time'].diff().dt.total_seconds().fillna(0)
start_time_diff = validation_data.groupby('msisdn')['start_time'].diff().dt.total_seconds().fillna(0).reset_index(drop=True)
validation_data['start_time_diff'] = start_time_diff.copy()

In [58]:
train_labels

,msisdn,is_sa
0,1109993,1
1,1017493,1
2,1114848,1
3,1243192,1
4,1270247,1
...,...,...
26687,1303962_4,0
26688,1304080_1,0
26689,1304080_2,0
26690,1304080_3,0


数据特征处理

In [59]:
# 聚合特征
def aggregate_features(data):
    return data.groupby('msisdn').agg({
    'call_duration': [
        # ('call_duration_sum', 'sum'), 
        ('call_duration_mean', 'mean'), 
        ('call_duration_max', 'max'), 
        ('call_duration_std', 'std'),
        ('call_duration_quantile_25', lambda x: x.quantile(0.25)), 
        ('call_duration_quantile_50', lambda x: x.quantile(0.50)), 
        ('call_duration_quantile_75', lambda x: x.quantile(0.75))
    ],
    'cfee': [
        # ('cfee_sum', 'sum'),
        ('cfee_std', 'std'), 
        ('cfee_mean', 'mean'),
    ],
    'lfee': [
        # ('lfee_sum', 'sum'), 
        ('lfee_mean', 'mean'),
        ('lfee_std', 'std'),
    ],
    'hour': [
        ('hour_mean', 'mean'), 
        ('hour_std', 'std'), 
        # ('hour_max', 'max'), 
        ('hour_min', 'min'),
    ],
    'dayofweek': [
        ('dayofweek_std', 'std'), 
        ('magic_dayofweek', lambda x: x.value_counts().mean()), 
        # ('work_day_num', lambda x: x[x.isin([1,2,3,4,5])].count()), 
        # ('weekend_num', lambda x: x[x.isin([6,7])].count()),
        ('dayofweek_mode', lambda x: x.mode().values[0]),
        ('work_day_weekend_diff', lambda x: (x[x.isin([1,2,3,4,5])].count() - x[x.isin([6,7])].count()) / (x[x.isin([1,2,3,4,5])].count() + x[x.isin([6,7])].count())),
    ],
    # 'home_area_code': [
    #     ('home_area_code_nunique', 'nunique')
    # ],
    'visit_area_code': [
        ('visit_area_code_nunique', 'nunique'),
        ('times_not_at_home_area', lambda x: x[x != x.shift()].count())
    ],
    'called_home_code': [
        ('called_home_code_nunique', 'nunique'),
        ('called_diff_home_code', lambda x: x[x != x.shift()].count())
    ],
    'called_code': [
        # ('called_code_nunique', 'nunique')
        ('called_code_diff', lambda x: x[x != x.shift()].count())
    ],
    'open_datetime': [
        ('open_count', 'nunique')
    ],
    'other_party': [
        ('account_person_num', 'nunique'),
        ('called_diff_home_code', lambda x: x[x != x.shift()].count())
    ],
    'a_serv_type': [
        # ('call_num', lambda x: x[x.isin([1, 3])].count()), 
        # ('called_num', lambda x: x[x == 2].count()),
        ('call_called_normalized_diff', lambda x: (x[x.isin([1, 3])].count() - x[x == 2].count()) /  (x[x.isin([1, 3])].count() + x[x == 2].count())),
    ],
    'start_time_diff': [
        ('start_time_diff_mean', 'mean'), 
        ('start_time_diff_std', 'std'), 
        ('start_time_diff_max', 'max'), 
    ], 
    'distance': [
        # ('distance_sum', 'sum'), 
        ('distance_std', 'std'), 
        # ('distance_max', 'max'), 
        # ('distance_quantile_25', lambda x: x.quantile(0.25)), 
        ('distance_quantile_50', lambda x: x.quantile(0.50)), 
        ('distance_quantile_75', lambda x: x.quantile(0.75)),
    ]
})

train_features = aggregate_features(train_data)
validation_features = aggregate_features(validation_data)


train_features.columns = ['_'.join(col).strip() for col in train_features.columns.values]
validation_features.columns = ['_'.join(col).strip() for col in validation_features.columns.values]
train_features.columns = train_features.columns.str.replace('[', '').str.replace(']', '').str.replace('<', '').str.replace('>', '').str.replace('(', '').str.replace(')', '').str.replace(',', '').str.replace(' ', '_')
validation_features.columns = validation_features.columns.str.replace('[', '').str.replace(']', '').str.replace('<', '').str.replace('>', '').str.replace('(', '').str.replace(')', '').str.replace(',', '').str.replace(' ', '_')
# 重置索引
train_features = train_features.reset_index()
validation_features = validation_features.reset_index()

# 合并标签数据
train_features = train_features.merge(train_labels, on='msisdn', how='left')
# 打印结果
# train_features

In [60]:
y = train_features['is_sa']
X = train_features.drop(['msisdn', 'is_sa'], axis=1)
X_validation = validation_features.drop(['msisdn'], axis=1)

n_sample = y.shape[0]
n_pos_sample = y[y ==1].shape[0]
n_neg_sample = y[y == 0].shape[0]
print('样本个数：{}; 正样本占{:.2%}; 负样本占{:.2%}'.format(n_sample,
                                                   n_pos_sample / n_sample,
                                                   n_neg_sample / n_sample))
print('特征维数：', X.shape[1])

样本个数：26692; 正样本占43.59%; 负样本占56.41%
特征维数： 32


In [61]:
# TODO use all_X to impute
imputer = SimpleImputer(strategy='most_frequent')
X = imputer.fit_transform(X)

In [62]:
imputer2 = SimpleImputer(strategy='most_frequent')
X_validation = imputer2.fit_transform(X_validation)

In [63]:
from sklearn.utils import shuffle
if ALL:
    if not NO_SMOTE:
        smote = SMOTE(random_state=42)    # 处理过采样的方法
        X, y = smote.fit_resample(X, y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42, shuffle=True)
    train_len = len(y_train) + len(y_test)
    test_len = 0
else:
    # X_train,X_test,y_train,y_test = train_test_split(X,y,stratify = y,test_size= 0.3,random_state=42, shuffle=True)

    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size= 0.2,random_state=42, shuffle=False)
    X_train, y_train = shuffle(X_train, y_train, random_state=42)
    X_test, y_test = shuffle(X_test, y_test, random_state=42)

    if not NO_SMOTE:
        smote = SMOTE(random_state=42)    # 处理过采样的方法
        X_train, y_train = smote.fit_resample(X_train, y_train)
        print('通过SMOTE方法平衡正负样本后')
        n_sample = y_train.shape[0]
        n_pos_sample = y_train[y_train == 1].shape[0]
        n_neg_sample = y_train[y_train == 0].shape[0]
        print('样本个数：{}; 正样本占{:.2%}; 负样本占{:.2%}'.format(n_sample,
                                                        n_pos_sample / n_sample,
                                                        n_neg_sample / n_sample))
        print('特征维数：', X.shape[1])
    train_len, test_len = len(y_train), len(y_test)

In [64]:
X_test.shape

(5339, 32)

In [65]:
print(y_test.value_counts())

is_sa
0    2759
1    2580
Name: count, dtype: int64


In [66]:
X.shape

(26692, 32)

In [67]:
columns = train_features.columns.tolist()
columns.remove('msisdn')

In [68]:
assert X_validation.shape[1] == X_train.shape[1]

In [69]:
# 拼接 X_train 和 y_train np.array 为 dataframe
train_set = np.c_[X_train, y_train]
train_set = pd.DataFrame(train_set, columns=columns)
test_set = np.c_[X_test, y_test]
test_set = pd.DataFrame(test_set, columns=columns)
valid_set = np.c_[X_validation, np.zeros(X_validation.shape[0])]
valid_set = pd.DataFrame(valid_set, columns=columns)
valid_set['is_sa'] = -1

In [70]:
test_set.describe()

,call_duration_call_duration_mean,call_duration_call_duration_max,call_duration_call_duration_std,call_duration_call_duration_quantile_25,call_duration_call_duration_quantile_50,call_duration_call_duration_quantile_75,cfee_cfee_std,cfee_cfee_mean,lfee_lfee_mean,lfee_lfee_std,...,other_party_account_person_num,other_party_called_diff_home_code,a_serv_type_call_called_normalized_diff,start_time_diff_start_time_diff_mean,start_time_diff_start_time_diff_std,start_time_diff_start_time_diff_max,distance_distance_std,distance_distance_quantile_50,distance_distance_quantile_75,is_sa
count,5339.000000,5339.000000,5339.000000,5339.000000,5339.000000,5339.000000,5339.000000,5339.000000,5339.000000,5339.000000,...,5339.000000,5339.000000,5339.000000,5339.000000,5339.000000,5.339000e+03,5.339000e+03,5.339000e+03,5.339000e+03,5339.000000
mean,97.267718,1004.025286,153.443302,22.525988,46.065555,104.026409,4.432620,2.763389,0.169595,0.333330,...,58.713242,114.283012,0.246388,18689.023547,38808.267410,1.969628e+05,3.066638e+05,2.661770e+05,4.017378e+05,0.483237
std,71.878984,986.568533,139.754962,10.990861,27.233434,90.396097,12.502585,7.775191,2.037858,3.375661,...,109.993554,161.831174,0.480907,21564.033150,41068.580275,1.821195e+05,2.887492e+05,4.842896e+05,5.894852e+05,0.499766
min,7.500000,9.000000,0.707107,3.000000,7.000000,9.000000,0.000000,0.000000,-7.384615,0.000000,...,1.000000,1.000000,-1.000000,0.000000,63.376672,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
25%,53.785714,358.000000,64.645560,16.000000,29.000000,57.750000,0.000000,0.000000,0.000000,0.000000,...,14.500000,25.000000,-0.085714,5155.060465,12508.476796,7.375000e+04,3.972435e+04,0.000000e+00,0.000000e+00,0.000000
50%,78.617647,716.000000,114.344540,20.000000,39.500000,82.500000,0.000000,0.000000,0.000000,0.000000,...,29.000000,51.000000,0.187032,12182.475248,26381.519435,1.485410e+05,2.392620e+05,0.000000e+00,7.919600e+04,0.000000
75%,115.824642,1365.000000,195.391227,26.750000,56.500000,123.000000,0.000000,0.000000,0.000000,0.000000,...,58.000000,145.000000,0.644068,23444.769231,49166.788143,2.564360e+05,4.982600e+05,3.248530e+05,6.292165e+05,1.000000
max,665.446154,7201.000000,1464.633034,163.250000,307.000000,1016.000000,131.124501,58.014706,29.179487,44.402101,...,1294.000000,1521.000000,1.000000,402011.000000,692876.845683,1.202073e+06,2.047526e+06,3.735311e+06,3.735311e+06,1.000000


In [71]:
all_set = pd.concat([train_set, test_set, valid_set], axis=0).reset_index(drop=True)
labeled_data_len = train_set.shape[0] + test_set.shape[0]

In [72]:
test_set.shape, train_set.shape, valid_set.shape, all_set.shape

((5339, 33), (21353, 33), (1278, 33), (27970, 33))

In [73]:
labeled_set, valid_set = all_set.iloc[:labeled_data_len].copy(), all_set.iloc[labeled_data_len:].copy()
labeled_set.reset_index(drop=True, inplace=True)
valid_set.reset_index(drop=True, inplace=True)
# 有一些值在SMOTE后对数变换后为 NaN，需要删除这些数据
print(labeled_set.isnull().sum().sum())
labeled_set = labeled_set.dropna()
print(labeled_set.isnull().sum().sum())
assert valid_set.shape[0] == validation_features.shape[0]

0
0


In [74]:
# 重新划分训练集和测试集
if not ALL:
    train_set, test_set = labeled_set.iloc[:train_len].copy(), labeled_set.iloc[train_len:].copy()
    train_set.reset_index(drop=True, inplace=True)
    test_set.reset_index(drop=True, inplace=True)
    # # 从 test_set 和 y_test 中删除 数据增强获得的数据 无法实现
    # assert y.shape[0] == train_features.shape[0]
    # assert labeled_set.shape[0] == y.shape[0]
    # id_col = train_features['msisdn']
    # train_set_ids = id_col.iloc[train_set.index]
    # test_set_ids = id_col.iloc[test_set.index]

In [75]:
# 使用 autogluon 训练
from autogluon.tabular import TabularPredictor
# 输入数据X_train, y_train
if not ALL:
    model = TabularPredictor(label='is_sa', eval_metric='f1', problem_type='binary').fit(train_set, presets='medium_quality')
    # model = TabularPredictor(label='is_sa', eval_metric='f1', problem_type='binary').fit(train_set, presets='best_quality', time_limit=3600)
else:
    model = TabularPredictor(label='is_sa', eval_metric='f1', problem_type='binary').fit(labeled_set, presets='best_quality', time_limit=3600)

No path specified. Models will be saved in: "AutogluonModels\ag-20240719_153611"
Presets specified: ['medium_quality']
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20240719_153611"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19041
CPU Count:          12
Memory Avail:       21.03 GB / 31.90 GB (65.9%)
Disk Space Avail:   487.58 GB / 3726.01 GB (13.1%)
Train Data Rows:    21353
Train Data Columns: 32
Label Column:       is_sa
Problem Type:       binary
Preprocessing data ...
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    21530.42 MB
	Train Data (Original)  Memory Usage: 5.21 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set 

In [76]:
if not ALL:
    print(model.evaluate(test_set))

{'f1': 0.835820895522388, 'accuracy': 0.8598988574639446, 'balanced_accuracy': 0.8559440421903228, 'mcc': 0.7367845717904491, 'roc_auc': 0.948917847439388, 'precision': 0.9635627530364372, 'recall': 0.737984496124031}


In [77]:
feature_importance = model.feature_importance(test_set if not ALL else labeled_set)
print(feature_importance)
feature_importance

Computing feature importance via permutation shuffling for 32 features using 5000 rows with 5 shuffle sets...
	32.99s	= Expected runtime (6.6s per shuffle set)
	12.99s	= Actual runtime (Completed 5 of 5 shuffle sets)


                                           importance    stddev       p_value  \
other_party_called_diff_home_code            0.068371  0.001717  4.767571e-08   
called_home_code_called_home_code_nunique    0.065911  0.006841  1.372915e-05   
a_serv_type_call_called_normalized_diff      0.029002  0.002973  1.306399e-05   
start_time_diff_start_time_diff_std          0.020472  0.004520  2.676416e-04   
start_time_diff_start_time_diff_max          0.020262  0.001864  8.495198e-06   
hour_hour_std                                0.019460  0.002149  1.756390e-05   
other_party_account_person_num               0.019353  0.002115  1.685438e-05   
cfee_cfee_mean                               0.012134  0.001388  2.021515e-05   
call_duration_call_duration_quantile_25      0.010606  0.001177  1.791104e-05   
dayofweek_dayofweek_std                      0.008149  0.003361  2.804992e-03   
hour_hour_min                                0.007344  0.001961  5.567000e-04   
distance_distance_quantile_5

,importance,stddev,p_value,n,p99_high,p99_low
other_party_called_diff_home_code,0.068371,0.001717,4.767571e-08,5,0.071906,0.064836
called_home_code_called_home_code_nunique,0.065911,0.006841,1.372915e-05,5,0.079996,0.051825
a_serv_type_call_called_normalized_diff,0.029002,0.002973,1.306399e-05,5,0.035124,0.022881
start_time_diff_start_time_diff_std,0.020472,0.004520,2.676416e-04,5,0.029780,0.011164
start_time_diff_start_time_diff_max,0.020262,0.001864,8.495198e-06,5,0.024100,0.016424
hour_hour_std,0.019460,0.002149,1.756390e-05,5,0.023885,0.015035
other_party_account_person_num,0.019353,0.002115,1.685438e-05,5,0.023709,0.014998
cfee_cfee_mean,0.012134,0.001388,2.021515e-05,5,0.014992,0.009275
call_duration_call_duration_quantile_25,0.010606,0.001177,1.791104e-05,5,0.013029,0.008182
dayofweek_dayofweek_std,0.008149,0.003361,2.804992e-03,5,0.015068,0.001229


In [78]:
# leaderboard
if not ALL:
    leaderboard = model.leaderboard(test_set, silent=True)
    print(leaderboard)
else:
    leaderboard = model.leaderboard(labeled_set, silent=True)
    print(leaderboard)
leaderboard

                  model  score_test  score_val eval_metric  pred_time_test  \
0   WeightedEnsemble_L2    0.835821   0.998896          f1        0.169564   
1               XGBoost    0.834140   0.997239          f1        0.047395   
2              LightGBM    0.831341   0.997792          f1        0.047268   
3              CatBoost    0.827662   0.997792          f1        0.069347   
4       NeuralNetFastAI    0.822217   0.997792          f1        0.104900   
5            LightGBMXT    0.821586   0.997792          f1        0.033813   
6         LightGBMLarge    0.810787   0.997239          f1        0.055044   
7        NeuralNetTorch    0.803779   0.997245          f1        0.063404   
8      RandomForestEntr    0.798996   0.997239          f1        0.123326   
9      RandomForestGini    0.790730   0.997239          f1        0.147900   
10       ExtraTreesEntr    0.778893   0.997236          f1        0.140020   
11       ExtraTreesGini    0.771220   0.997236          f1      

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.835821,0.998896,f1,0.169564,0.080749,210.150338,0.003000,0.005517,3.507675,2,True,14
1,XGBoost,0.834140,0.997239,f1,0.047395,0.018160,2.530426,0.047395,0.018160,2.530426,1,True,11
2,LightGBM,0.831341,0.997792,f1,0.047268,0.022000,3.948582,0.047268,0.022000,3.948582,1,True,4
3,CatBoost,0.827662,0.997792,f1,0.069347,0.011631,150.458392,0.069347,0.011631,150.458392,1,True,7
4,NeuralNetFastAI,0.822217,0.997792,f1,0.104900,0.033000,20.433728,0.104900,0.033000,20.433728,1,True,10
5,LightGBMXT,0.821586,0.997792,f1,0.033813,0.024518,3.228809,0.033813,0.024518,3.228809,1,True,3
6,LightGBMLarge,0.810787,0.997239,f1,0.055044,0.025910,4.164466,0.055044,0.025910,4.164466,1,True,13
7,NeuralNetTorch,0.803779,0.997245,f1,0.063404,0.039084,52.955463,0.063404,0.039084,52.955463,1,True,12
8,RandomForestEntr,0.798996,0.997239,f1,0.123326,0.100977,2.795939,0.123326,0.100977,2.795939,1,True,6
9,RandomForestGini,0.790730,0.997239,f1,0.147900,0.067829,2.092542,0.147900,0.067829,2.092542,1,True,5


In [79]:
# 在testset 上计算指标
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

if not ALL:
    y_pred = model.predict(test_set)
    y_true = test_set['is_sa']
    print(classification_report(y_true, y_pred))
    print(confusion_matrix(y_true, y_pred))

              precision    recall  f1-score   support

         0.0       0.80      0.97      0.88      2759
         1.0       0.96      0.74      0.84      2580

    accuracy                           0.86      5339
   macro avg       0.88      0.86      0.86      5339
weighted avg       0.88      0.86      0.86      5339

[[2687   72]
 [ 676 1904]]


In [80]:
valid_set

,call_duration_call_duration_mean,call_duration_call_duration_max,call_duration_call_duration_std,call_duration_call_duration_quantile_25,call_duration_call_duration_quantile_50,call_duration_call_duration_quantile_75,cfee_cfee_std,cfee_cfee_mean,lfee_lfee_mean,lfee_lfee_std,...,other_party_account_person_num,other_party_called_diff_home_code,a_serv_type_call_called_normalized_diff,start_time_diff_start_time_diff_mean,start_time_diff_start_time_diff_std,start_time_diff_start_time_diff_max,distance_distance_std,distance_distance_quantile_50,distance_distance_quantile_75,is_sa
0,60.308682,1700.0,112.715262,19.50,37.0,62.50,0.000000,0.00,0.0,0.0,...,69.0,249.0,-0.421222,4071.543408,8323.861851,51194.0,2.081493e+05,0.0,180274.0,-1.0
1,54.165854,362.0,56.854232,17.00,37.0,65.00,0.000000,0.00,0.0,0.0,...,72.0,174.0,-0.160976,6229.775610,10913.381674,51724.0,2.102743e+04,0.0,0.0,-1.0
2,89.500000,670.0,175.297814,12.50,37.5,49.00,0.000000,0.00,0.0,0.0,...,8.0,14.0,-0.333333,65299.611111,72201.374485,217652.0,0.000000e+00,0.0,0.0,-1.0
3,46.458515,399.0,58.811953,14.00,27.0,53.00,0.000000,0.00,0.0,0.0,...,27.0,155.0,0.362445,5491.794760,11149.929456,65807.0,0.000000e+00,231682.0,231682.0,-1.0
4,38.430556,246.0,46.076735,13.00,19.0,42.50,6.804617,3.75,0.0,0.0,...,44.0,51.0,0.500000,16717.666667,39100.045311,242220.0,1.696337e+05,29847.5,280421.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1273,156.426230,1804.0,318.203739,23.00,45.0,96.50,0.000000,0.00,0.0,0.0,...,55.0,144.0,0.136612,6796.404372,17418.083154,148767.0,7.154820e+04,0.0,0.0,-1.0
1274,52.897872,621.0,69.502307,19.00,34.0,54.00,0.000000,0.00,0.0,0.0,...,36.0,130.0,0.055319,5298.587234,13151.545900,144098.0,1.749472e+05,0.0,0.0,-1.0
1275,346.552448,4061.0,637.773302,34.00,96.0,389.50,0.000000,0.00,0.0,0.0,...,23.0,114.0,0.230769,8752.293706,15958.753297,72131.0,3.078249e+05,0.0,0.0,-1.0
1276,22.500000,44.0,30.405592,11.75,22.5,33.25,0.000000,0.00,0.0,0.0,...,2.0,2.0,0.000000,91577.000000,129509.435401,183154.0,1.117818e+06,790417.0,1185625.5,-1.0


In [81]:
# 预测
y_validation_pred = model.predict(valid_set.drop('is_sa', axis=1))

# 将预测结果与 msisdn 对应起来
validation_results = validation_features[['msisdn']].copy()
validation_results['is_sa'] = y_validation_pred.astype(int)

print(validation_results.describe())

# 保存结果到CSV文件
file_name = './valid_large_data.csv' if ALL else './valid_small_data.csv'
validation_results.to_csv(file_name, index=False)
print(file_name)

             is_sa
count  1278.000000
mean      0.174491
std       0.379680
min       0.000000
25%       0.000000
50%       0.000000
75%       0.000000
max       1.000000
./valid_small_data.csv
